# Multivariate Linear Regression & Preprocessing
فكّر فيها بإنها مهمة وضع ثِقل لكل خاصية مستقلة بناءا علي مدي قوة تأثيرها علي المتغير تحت الرصد المُراد توقّعه

# Problem Definition: 
### The dataset contains detailed attributes for every player registered in the latest edition of FIFA 19 database. Our goal is to create Linear and Polynomial Regression models to predict the value of the player based on several attributes. 


In [14]:
import sys
sys.path.append('/home/ahmed/FCIS-Seniority/pattern-recognition/modules')

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# 
from sklearn import linear_model
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
# 
from pre_processing import features_encoder
from pre_processing import features_scaling


In [4]:
fifa_df = pd.read_csv('./fifa19.csv')
fifa_df.head()

,Name,Age,Nationality,Overall,Potential,Club,Position,Crossing,Finishing,HeadingAccuracy,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,Value
0,L. Messi,31,Argentina,94,94,FC Barcelona,RF,84.0,95.0,70.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,110.5
1,Cristiano Ronaldo,33,Portugal,94,94,Juventus,ST,84.0,94.0,89.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,77.0
2,Neymar Jr,26,Brazil,92,93,Paris Saint-Germain,LW,79.0,87.0,62.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,118.5
3,De Gea,27,Spain,91,93,Manchester United,GK,17.0,13.0,21.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,72.0
4,K. De Bruyne,27,Belgium,91,92,Manchester City,RCM,93.0,82.0,55.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,102.0


In [5]:
fifa_df.columns

Index(['Name', 'Age', 'Nationality', 'Overall', 'Potential', 'Club',
       'Position', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing',
       'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing',
       'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions',
       'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving',
       'GKHandling', 'GKKicking', 'GKPositioning', 'Value'],
      dtype='object')

In [7]:
print(f'No. of Players: {len(fifa_df)}')

No. of Players: 17955


In [8]:
fifa_df.describe()

,Age,Overall,Potential,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,Value
count,17955.000000,17955.000000,17955.000000,17907.000000,17907.000000,17907.000000,17907.000000,17907.000000,17907.000000,17907.000000,...,17907.000000,17907.000000,17907.000000,17907.000000,17907.000000,17907.000000,17907.000000,17907.000000,17907.000000,17955.000000
mean,25.095349,66.228237,71.323197,49.751885,45.586698,52.298878,58.717261,42.935500,55.420283,47.218630,...,48.546043,58.651868,47.262467,47.684202,45.644441,16.587536,16.365276,16.203105,16.356062,2.444530
std,4.659778,6.921015,6.139378,18.351445,19.511503,17.361599,14.673721,17.685432,18.899100,18.384605,...,15.685745,11.421007,19.872750,21.647299,21.271215,17.662332,16.875398,16.467295,16.989954,5.626715
min,16.000000,46.000000,48.000000,5.000000,2.000000,4.000000,7.000000,4.000000,4.000000,6.000000,...,5.000000,3.000000,3.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,0.010000
25%,21.000000,62.000000,67.000000,38.000000,30.000000,44.000000,54.000000,30.000000,49.000000,34.000000,...,39.000000,51.000000,30.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,0.325000
50%,25.000000,66.000000,71.000000,54.000000,49.000000,56.000000,62.000000,44.000000,61.000000,49.000000,...,49.000000,60.000000,53.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,0.700000
75%,28.000000,71.000000,75.000000,64.000000,62.000000,64.000000,68.000000,57.000000,68.000000,62.000000,...,60.000000,67.000000,64.000000,66.000000,64.000000,14.000000,14.000000,14.000000,14.000000,2.100000
max,45.000000,94.000000,95.000000,93.000000,95.000000,94.000000,93.000000,90.000000,97.000000,94.000000,...,92.000000,96.000000,94.000000,93.000000,91.000000,90.000000,92.000000,91.000000,90.000000,118.500000


In [9]:
fifa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17955 entries, 0 to 17954
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             17955 non-null  object 
 1   Age              17955 non-null  int64  
 2   Nationality      17955 non-null  object 
 3   Overall          17955 non-null  int64  
 4   Potential        17955 non-null  int64  
 5   Club             17955 non-null  object 
 6   Position         17907 non-null  object 
 7   Crossing         17907 non-null  float64
 8   Finishing        17907 non-null  float64
 9   HeadingAccuracy  17907 non-null  float64
 10  ShortPassing     17907 non-null  float64
 11  Volleys          17907 non-null  float64
 12  Dribbling        17907 non-null  float64
 13  Curve            17907 non-null  float64
 14  FKAccuracy       17907 non-null  float64
 15  LongPassing      17907 non-null  float64
 16  BallControl      17907 non-null  float64
 17  Acceleration

In [12]:
# drop rows that contains at least one na on a new Dataframe Instance
# what about tring fillna(value= , method= , inplace=False)
fifa_df_drop_na = fifa_df.dropna(axis='index', how='any', inplace=False)